In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
def init():
    preprocess_data = pd.read_pickle('./preprocessed_alcohol_v3.pkl')
    Glass_Dict = dict()
    measure = []
    with open('../glass_pre.txt', 'r') as file:
        for line in file:
            data = line.strip().split(' : ')
            Glass_Dict[data[0].lower()]=float(data[1])
    # print(Glass_Dict)
    glass_list = list()
    for glass in preprocess_data.Glass:
        glass_list.append(Glass_Dict[glass.lower()])
    preprocess_data['Glass_measure'] = glass_list
    return preprocess_data



In [182]:
conversion_ml_factors = {
    'ml': 1.0,
    'oz': 29.5735,
    'cl': 10.0,
    'tsp': 4.92892,
    'tblsp': 14.7868,
    'tablespoon': 14.7868,
    'tbsp': 14.7868,
    'dash':0.625,
    'cup': 236.588,
    'pinch': 0.31,
    'shot': 44.3603,
    'glasses':256.3,
    'drop': 0.05,
    'pint': 473.176,
    'can': 330.0,
    'jigger': 44.3603,
    'dl':100,
    'bottle': 750.0,
    'pint':473.176,
    'quart': 946.353,
    'qt': 946.353,
    'l': 1000.0,
    'splash': 29.5735,
    'gal':3785.41,
    'dash_of': 0.92,
    'scoop': 45.0,
    'scoops':45.0,
    'handful': 60.0,
    'piece': 30.0,
    'juice of':40.0,
    'fifth':750
}
# Define the category synonyms
category_synonyms = {
    'fill': ['fill to top', 'fill up to', 'fill', 'to fill', 'top up with'],
    'cubes': ['cubes'],
    'garnish': ['garnish', 'twist of'],
    'a_little': ['a little bit of','pinch'],
    'splash': ['splash'],
    'dash': ['dash'],
    'wedge': ['wedge'],
    'bottle': ['bottle','large'],
    'pinch': ['pinch'],
    'taste': ['by taste', 'to taste']
}
with open('../category.json') as f:
    category_json = json.load(f)
with open('../flavor.json') as f:
    flavor_json = json.load(f)
# Initialize categories with empty lists to allow creation of new entries
categories = {key: set() for key in category_synonyms}
categories_D_name = {key: set() for key in category_synonyms}


In [172]:
def convert_volume(volume, unit):
    if unit in conversion_ml_factors:
        return volume * conversion_ml_factors[unit] , 'ml'
    else:
        return volume, unit

def convert_weight(IName,weight,unit):
    conversion_factors={
        'lb':450,
        'kg':1000,
        'package':300,
        'gr':1
    }
    if unit in conversion_factors:
        return weight * conversion_factors[unit],'gram'
    else:
        if IName == 'ice':
            return weight*30,'gram'
        elif IName == 'sugar':
            return weight*3,'gram'
        return weight,unit
def convert_garnish(unit):
    return None

# garnish : 'chunk','wedge','leaves','twist of','slice'
#glass에 따라 계산해야하는 단위 : 'glass','part'
# oz_to_ml(volume) = volume * 29.5735

In [ ]:
ice_list=['cubes']
fill_list =['fill to top', 'fill up to', 'fill', 'to fill', 'top up with']
garnish_list=['garnish', 'twist of']
a_little ='a little bit of' 
splash = 'splash', 
dash = 'dash', 
wedge='wedge'
large = 'large'
bottle = 'bottle'
pinch = 'pinch'
taste = ['by taste', 'to taste']

'1/4', 
'1/6',  
'1/3', 
'1/5', 
'2/3', 
'2/5', 
'1/2', 
'3/4'

one_ing_categories = {
    'ice': set(),
    'fill': set(),
    'garnish': set(),
    'a_little': set(),
    'splash': set(),
    'dash': set(),
    'wedge': set(),
    'large': set(),
    'bottle': set(),
    'pinch': set(),
    'taste': set()
}

In [53]:
category_synonyms['fill']

['fill to top', 'fill up to', 'fill', 'to fill', 'top up with']

In [169]:
preprocess_data = init()

In [ ]:
import copy
# fill 처리
# fill이 들어가면, 해당 재료 외에 다른 재료들을 확인하고, glass 용량에서 빼준다. 
for drink,ingredient,measures,glass_ml in zip(preprocess_data.DrinkName,preprocess_data.Ingredients,preprocess_data.Measures,preprocess_data.Glass_measure):
    for i in range(len(ingredient)):
        Iname,order = ingredient[i]
        if len(measures[i][0])==1:
            unit = measures[i][0]
            
            if unit[0] in category_synonyms['fill']:
                print(glass_ml)
                glass_max = copy.deepcopy(glass_ml)
                for j in range(len(ingredient)):
                    if i!=j:
                        Iname2,order2 = ingredient[j]
                        print(measures[j][0])
                        M,unit2 = measures[j][0]
                        if unit2 in list(conversion_ml_factors.keys()):
                            volume = convert_volume(M,unit2)
                            glass_max -= volume
                print(f'{drink},{Iname},{glass_ml},{glass_max}')

            


In [ ]:
#Glass 정보로 전처리
# 확인이 필요한 재료정보 -> ml로 변경해야함
last_num = list()
for drink,ingredient,measures,glass in zip(preprocess_data.DrinkName,preprocess_data.Ingredients,preprocess_data.Measures,preprocess_data.Glass):
    #재료수
    i_num = len(ingredient)
    m_num = len(measures)
    last_num.append(i_num-1)

In [114]:
# 3-Mile Long Island Iced Tea 레시피 잘못됨
# coca-cola <->bitters, lemon<->sweet and sour 바뀜

A = preprocess_data.loc[preprocess_data.DrinkName=='3-Mile Long Island Iced Tea']



8    [[gin, 1], [light rum, 2], [tequila, 3], [trip...
Name: Ingredients, dtype: object


In [ ]:
preprocess_data = init()
TEST_DATA = copy.deepcopy(preprocess_data)
TEST_DATA

In [199]:
preprocess_data = init()
TEST_DATA = copy.deepcopy(preprocess_data)
TEST_DATA['Garnish']=0
# TEST_DATA['Ingredient_List']=None
# TEST_DATA['Measure_List']=None
# print(last_num)
num_one_list=list()
test_garnish_set = set()
part_set = set()
glass_set = set()
Garnish_List = ['garnish','slice', 'sprig', 'wedge', 'twist of', 'long strip',
                'chunk','chunks','stick', 'leaves','long strip']
for DName, ingredient,measures,glass in zip(TEST_DATA.DrinkName, TEST_DATA.Ingredients, TEST_DATA.Measures, TEST_DATA.Glass_measure):
    #아래 항목은 건너뛴다. large의 의미가 좀 이상함
    if DName in ['3-Mile Long Island Iced Tea']:
        continue
    for i in range(len(ingredient)):
        Iname,order = ingredient[i]
        R=0
        if len(measures[i][0])==1:
            convert_unit = measures[i][0]
            continue
        elif len(measures[i][0])==2:
            M,unit = measures[i][0]
            convert_unit = unit
            if unit=='ml' or unit=='gram':
                continue
            R = M
            if (isinstance(M, str) and 'juice of'in M):
                tmp_unit = copy.deepcopy(M)
                M = copy.deepcopy(unit)
                unit = copy.deepcopy(tmp_unit)
                R = M
                if('/' in M):
                    a,b = M.split('/')
                    R = float(a)/float(b)
            elif('whole' in unit):
                if(Iname=='egg yolk'):
                    R = 17
                    unit = 'ml'
                elif(Iname=='egg white'):
                    R = 30
                    unit = 'ml'
                elif(Iname=='egg'):
                    R = 47
                    unit = 'ml'
                elif(Iname=='lemon'):
                    R = 40
                    unit = 'ml'
                else:
                    continue
                
            elif(isinstance(M, str))and(('/' in M) or ('-' in M)):
                if('/' in M):
                    a,b = M.split('/')
                    R = float(a)/float(b)
                elif('-' in M):
                    a,b = M.split('-')
                    R = (float(a) + float(b))/2
            elif(isinstance(M, str))and('/' in unit):
                # num, num case
                a,b = unit.split('/')
                N = float(a)/float(b)
                R = float(M) + N
                if(Iname=='light rum'):
                    unit='oz'
                else:
                    print(f"{Iname} : {R} {unit} , original unit:{M} {unit}")
                    continue
            
            R,convert_unit = convert_volume(float(R), unit)
            if convert_unit=='ml':
                measures[i][0] = [R,convert_unit]
            else:
                R,convert_unit=convert_weight(Iname,float(R),unit)
                if convert_unit=='gram':
                    measures[i][0] = [R,convert_unit]

            if convert_unit=='ml' or convert_unit=='gram':
                continue
            else:
                test_garnish_set.add(convert_unit)
                if convert_unit=='part':
                    part_set.add(DName)
                elif convert_unit=='glass':
                    glass_set.add(DName)
                print(f"{Iname} : {R} {convert_unit} , original unit:{M} {unit}")
            # print(f"{Iname} : {R} {convert_unit} , original unit:{M} {unit}")
        elif len(measures[i][0])==3:
            M,N,L = measures[i][0]
            convert_unit = L
            if L=='ml' or L=='gram':
                continue
            # num, num, unit 
            if('/' in N):
                a,b = N.split('/')
                N = float(a)/float(b)
                R = float(M) + N
            elif('/' in M):
                a,b = M.split('/')
                M = float(a)/float(b)
                R = float(M) + float(N)
            
            else:
                R = float(M) + float(N)
                # print(f"{Iname} : {M} {N} {L}")
            R,convert_unit = convert_volume(float(R), L)
            if convert_unit=='ml':
                measures[i][0] = [R,convert_unit]
            else:
                R,convert_unit=convert_weight(Iname,float(R),L)
                if convert_unit=='gram':
                    measures[i][0] = [R,convert_unit]
            if convert_unit=='ml' or convert_unit=='gram':
                continue
            else:
                test_garnish_set.add(convert_unit)
                print(f"{Iname} : {R} {convert_unit} , original unit: {M} {N} {L}")
                if L=='part':
                    part_set.add(DName)
                elif L=='glass':
                    glass_set.add(DName)
    


kahlua : 0.3333333333333333 part , original unit:1/3 part
bailey's irish cream : 0.3333333333333333 part , original unit:1/3 part
frangelico : 0.3333333333333333 part , original unit:1/3 part
lemon peel : 1.0 twist of , original unit:1 twist of
lemon peel : 1.0 twist of , original unit:1 twist of
maraschino cherry : 1.0 garnish , original unit:1 garnish
absolut citron : 0.6666666666666666 part , original unit:2/3 part
pisang ambon : 0.3333333333333333 part , original unit:1/3 part
lemon : 1.0 slice , original unit:1 slice
mint : 1.0 leaves , original unit:1 leaves
rum : 1.0 part , original unit:1 part
vodka : 1.0 part , original unit:1 part
tequila : 1.0 part , original unit:1 part
triple sec : 0.5 part , original unit:1/2 part
lime : 1.0 wedge , original unit:1 wedge
grenadine : 1.0 part , original unit:1 part
orange juice : 4.0 part , original unit:4 part
pineapple juice : 4.0 part , original unit:4 part
frangelico : 1.5 part , original unit:3/2 part
lemon peel : 1.0 twist of , origi

In [201]:
part_set,glass_set

({'747',
  'Absolut Evergreen',
  'Adam Bomb',
  'Afterglow',
  'Afternoon',
  'Amaretto Stone Sour',
  'Amaretto Sweet & Sour',
  'Angelica Liqueur',
  'Apple Pie with A Crust',
  'Apple Slammer',
  'Arctic Fish',
  'Avalon',
  'Black & Tan',
  'Butter Baby',
  'Citrus Coke',
  'Danbooka',
  'Darkwood Sling',
  'Downshift',
  'Gagliardo',
  'Golden dream',
  'Grape lemon pineapple Smoothie',
  'Grizzly Bear',
  'Hemingway Special',
  'Homemade Kahlua',
  'Karsk',
  'Kir',
  'Kir Royale',
  'Kiwi Lemon',
  'Malibu Twister',
  'Nutty Irishman',
  'Paradise',
  "Planter's Punch",
  'Porto flip',
  'Pysch Vitamin Light',
  'Quick F**K',
  'Royal Bitch',
  'Shot-gun',
  'Smut',
  'Surf City Lifesaver',
  'Talos Coffee',
  'Texas Rattlesnake'},
 {'Bleeding Surgeon', 'Butter Baby', "Flander's Flake-Out", 'Jamaican Coffee'})

In [193]:
test_garnish_set
# TEST_DATA

{'chunk',
 'garnish',
 'glass',
 'inch',
 'large',
 'leaves',
 'long strip',
 'part',
 'slice',
 'sprig',
 'stick',
 'strig',
 'twist of',
 'wedge'}

In [124]:
TEST_DATA.Measures

0               [[[29.5735, ml], 1], [[29.5735, ml], 2]]
1      [[[14.78675, ml], 1], [[7.393375, ml], 2], [[7...
2              [[[473.176, ml], 1], [[44.36025, ml], 2]]
3      [[[14.78675, ml], 1], [[14.78675, ml], 2], [[1...
4      [[[44.3603, ml], 1], [[88.7206, ml], 2], [[44....
                             ...                        
541    [[[59.147, ml], 1], [[59.147, ml], 2], [[118.2...
542    [[[50.0, ml], 1], [[20.0, ml], 2], [[cubes], 3...
543    [[[330.0, ml], 4], [[1/6], 1], [[1/6], 2], [[1...
544             [[[1, 1/4, oz], 1], [[7.393375, ml], 2]]
545    [[[20.0, ml], 1], [[20.0, ml], 2], [[20.0, ml]...
Name: Measures, Length: 546, dtype: object